In [1]:
# --------------------------------------------
# 1️⃣ Configuración e importación de librerías
# --------------------------------------------
import pandas as pd
import numpy as np
from prophet import Prophet
from tqdm import tqdm  # Para barra de progreso
import os
from prophet import Prophet
import cmdstanpy


# Opcional: para visualizar más columnas
pd.set_option('display.max_columns', None)


In [2]:
# --------------------------------------------
# 2️⃣ Cargar dataset parquet
# --------------------------------------------

# Ruta local (ajusta si hace falta)
file_path = r"C:\Developer\Laboratorio_III\data\dataset_product_periodo.parquet"

df = pd.read_parquet(file_path)

print(f"✅ Dataset cargado con forma: {df.shape}")
df.head()


✅ Dataset cargado con forma: (31522, 13)


,product_id,periodo,tn_total,clientes_positivos,cat1,cat2,cat3,brand,sku_size,descripcion,fecha,mm-yyyy,quarter
0,20001,201701,934.77222,186,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-01-01,01-2017,2017Q1
1,20001,201702,798.01620,185,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-02-01,02-2017,2017Q1
2,20001,201703,1303.35771,188,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-03-01,03-2017,2017Q1
3,20001,201704,1069.96130,104,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-04-01,04-2017,2017Q2
4,20001,201705,1502.20132,238,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-05-01,05-2017,2017Q2


In [3]:
# --------------------------------------------
# 3️⃣ Verificar columnas clave
# --------------------------------------------
print(df.columns)

# Nos aseguramos de tener: product_id, fecha, tn_total
df[['product_id', 'fecha', 'tn_total']].head()


Index(['product_id', 'periodo', 'tn_total', 'clientes_positivos', 'cat1',
       'cat2', 'cat3', 'brand', 'sku_size', 'descripcion', 'fecha', 'mm-yyyy',
       'quarter'],
      dtype='object')


,product_id,fecha,tn_total
0,20001,2017-01-01,934.77222
1,20001,2017-02-01,798.01620
2,20001,2017-03-01,1303.35771
3,20001,2017-04-01,1069.96130
4,20001,2017-05-01,1502.20132


In [4]:
# --------------------------------------------
# 4️⃣ Inicializar lista para guardar resultados
# --------------------------------------------
results = []


In [5]:
# --------------------------------------------
# 5️⃣ Loop por producto: Prophet + extraer componentes
# --------------------------------------------

# Nos aseguramos que 'fecha' sea datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Iterar por product_id con barra de progreso
for pid, group in tqdm(df.groupby('product_id'), desc="Procesando productos"):
    try:
        # Ordenar por fecha
        group = group.sort_values('fecha')
        
        # Verificar datos suficientes
        if len(group) < 5:
            continue
            
        # Estructura para Prophet: ds, y
        df_prophet = pd.DataFrame({
            'ds': group['fecha'],
            'y': group['tn_total']
        })

        # Filtrar grupos problemáticos
        if df_prophet['y'].isnull().any() or df_prophet['y'].nunique() < 2:
            continue

        # Inicializar y entrenar Prophet (sin especificar backend)
        m = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=False,
            daily_seasonality=False
        )
        
        # Ajustar modelo
        m.fit(df_prophet)
        
        # Predicciones
        forecast = m.predict(df_prophet[['ds']])
        
        # Crear dataframe de componentes
        comp = pd.DataFrame({
            'product_id': pid,
            'fecha': forecast['ds'],
            'trend': forecast['trend'],
            'seasonal': forecast['seasonal'],
            'additive_terms': forecast['additive_terms'],
            'multiplicative_terms': forecast['multiplicative_terms']
        })
        
        # Agregar rangos
        comp['trend_max'] = forecast['trend'].max()
        comp['trend_min'] = forecast['trend'].min()
        comp['trend_range'] = comp['trend_max'] - comp['trend_min']
        comp['seasonal_max'] = forecast['seasonal'].max()
        comp['seasonal_min'] = forecast['seasonal'].min()
        comp['seasonal_range'] = comp['seasonal_max'] - comp['seasonal_min']
        
        results.append(comp)
        
    except Exception as e:
        print(f"❌ Error en producto {pid}: {str(e)}")
        continue

Procesando productos:   0%|          | 0/1233 [00:00<?, ?it/s]

Procesando productos:   0%|          | 3/1233 [00:00<00:54, 22.49it/s]

❌ Error en producto 20001: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20002: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20003: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20004: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20005: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20006: 'Prophet' object has no attribute 'stan_backend'


Procesando productos:   1%|          | 9/1233 [00:00<00:49, 24.57it/s]

❌ Error en producto 20007: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20008: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20009: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20010: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20011: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 20012: 'Prophet' object has no attribute 'stan_backend'


...

❌ Error en producto 21256: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21257: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21259: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21260: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21261: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21262: 'Prophet' object has no attribute 'stan_backend'


Procesando productos:  97%|█████████▋| 1200/1233 [00:43<00:01, 29.84it/s]

❌ Error en producto 21263: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21264: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21265: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21266: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21267: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21268: 'Prophet' object has no attribute 'stan_backend'


Procesando productos: 100%|██████████| 1233/1233 [00:43<00:00, 28.39it/s]

❌ Error en producto 21269: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21271: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21273: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21274: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21276: 'Prophet' object has no attribute 'stan_backend'
❌ Error en producto 21281: 'Prophet' object has no attribute 'stan_backend'


In [10]:
# --------------------------------------------
# 6️⃣ Unir todos los productos en un solo DataFrame
# --------------------------------------------
df_prophet_features = pd.concat(results, ignore_index=True)

print(f"✅ Dimensión final de variables Prophet: {df_prophet_features.shape}")
df_prophet_features.head()


ValueError: No objects to concatenate

In [ ]:
!pip install prophet==1.1.4 cmdstanpy==1.1.0 pandas numpy tqdm

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 9: invalid continuation byte